# SA-CCR

When using *SA-CCR* the exposure at default (EAD) has to be calculated as: 

\begin{align*}
EAD &= \alpha * (RC + PFE)\\
\\
\text{where} \qquad \alpha&=1.4 \\
RC&: \text{Replacement Cost} \\
PFE&: \text{Potential Future Exposure}
\end{align*}

Let's set this relationship up in Python

In [1]:
# Imports
from datetime import datetime, timedelta
from enum import Enum
from math import exp, log, sqrt
from typing import List
import numpy
from pandas import read_csv, DataFrame, Series
from scipy.stats import norm
from Enums import TradeType, TradeDirection, SwapDirection, AssetClass, MaturityBucket, SubClass, CreditSubClass, \
    EquitySubClass, Currency, CurrencyPair, Stock
from collateralAgreement import CollateralAgreement, Margining, Clearing, Tradecount, Dispute
from instruments.Trade import Trade
from instruments.equity_instruments.equityForward import EquityForward
from instruments.fx_instruments.fxForward import FxForward
from instruments.fx_instruments.fxOption import FxOption

def calculate_sa_ccr_ead(rc:float , pfe: float) -> float:
    """
    Calculate EAD of SA-CCR as defined in paragraph 186.
               
    :param rc: Replacement Cost
    :param pfe: Potential Future Exposure
    :return: Exposure at default according to SA-CCR
    """

    alpha = 1.4 #Carried over from alpha used for IMM
    return alpha*(rc+pfe)

calculate_sa_ccr_ead(10.0,15.0)

35.0

SA-CCR differentiates between the following asset classes:

* Interest rate derivatives
* Foreign exchange derivatives
* Credit derivatives
* Equity derivatives
* Commodity derivatives

Each of these asset classes is differentiated again into different hedging sets - Netting of expsoure is only possible within a common hedging set. The definition of a hedging set depends on the asset class. For FX derivatives derivatives based on the same currency pair form a hedging set. For Equity derivatives on the other hand all trades are in a common hedging set. For all asset classes, derivatives that have a basis as an underlying and volatility derivatives form two seperate hedging sets.

### Relation of RC and PFE
The purpose of the RC is to assess the imidiate loss suffered by the default of a counterparty. It is based on the current MtM of the derivative less the accessible collateral. If a bank has posted collateral to non-segregated accounts of a counterparty this collateral is also assumed to be lost in case of a default which increases the replacement cost.

The potential future exposure (PFE) on the other hand assesses how the RC might develop in the future. The future being defined as during the next year. If the RC today is 0 but is likely to be larger than 0 in the near future the estimated EAD should take this expected increase in RC into account.

See also Paragraph 130 and 131 of BCBS(2014):

Paragraph 130 - case without margining:

> For unmargined transactions, the *RC* intends to caputre the loss that would occur if a counterparty were to default and were closed out of its transactions immediately. The *PFE* add-on represents a potential conversative increase in expousre over a one-year time horizon from the present date (i.e. the calculation date).

Paragraph 131 - case with margining:

> For margined trades, the *RC* intends to capture the loss that would occur if a counterparty were to default at the present or at a future time, assuming that the closeout and replacement of transactions occur instantaneously. However, there may be a period (the margin period of risk) between the last exchange of collateral before default and replacement of the trades in the market. The *PFE* add-on represents the potential change in value of the trades during this time period.

## Definition of Potential Future Exposure (PFE)

\begin{align*}
PFE &= \text{multiplier} * AddOn^{\text{aggregate}} \\
\\
\text{where} \qquad AddOn^{\text{aggregate}} &: \text{aggregate add-on component} \\
\text{multiplier} &: f(V,C,AddOn^{\text{aggregate}})
\end{align*}

$AddOn$ is calculated differently for each asset $a$ class. Since no netting is allowed between asset classes the aggregate is calculated as:

$$AddOn^{\text{aggregate}} = \sum_{a}AddOn^{a}$$

Collateralization is taken into account of the PFE calculation through the multiplier that uses the collateral held as an input. As overcollateralization e.g. through IM increases, the multiplier decreases. However, the multiplier is floored at 5%.

\begin{align*}
\text{multiplier} &= \min \left\{ 1; Floor + (1-Floor) \exp\left(\frac{V-C}{2(1-Floor)AddOn^{\text{aggregate}}}\right) \right\} \\
\text{where} \qquad Floor &= 5\%
\end{align*}

In [2]:
def multiplier(v:float,c:float, addOn_aggregate:float, floor:float = 0.05) -> float:
    """
    Multiplier calculation for SA-CCR
    
    :param v: Current value of the derivative transactions in the netting set
    :param c: Haircut value of the net collateral held
    :param addOn_aggregate: Aggregated (summed up) AddOn over all asset classes
    :param floor: Regulatory floor for the multiplier. Set to 5% in paragraph 149
    :return: Multiplier for PFE calculation according to SA-CCR
    """
    
    return min(1, floor+(1-floor)*exp((v-c)/(2*(1-floor)*addOn_aggregate)))

# AddOn calculation

Most of the SA-CCR logic is hidden inside the AddOn calculation. At first it is important to define the following four data parameters:

#### $M_i$ 
> Maturity of the derivative contract. If the underlying of a derivative is another derivative - e.g. in the case of a swaption the maturity date of the underlying needs to be chosen.

#### $S_i$

> For interest rate and credit derivatives the start date of the time periodreferenced by an interst rate or credit contract. If the derivtives underlying is another interest rate or credit intsrument (eg swaption or bond option) $S_i$ is the start date of the underlying instead.

#### $E_i$

> Defined as $S_i$ but referencing the end date instead of the start date.

#### $T_i$

> For options across all asset classes this is the latest contractual exercise date.

Let's set up some trades at this point:

* An Equity Option that is defined by a strike, maturity and current value of the underlying. Additional characteristics e.g. if it is an american or european option are irrelevant for SA-CCR calculations
* A running intere rate swap
* A forward starting interest rate swap
* A swaption with the previous swap as underlying

In [3]:
from instruments.equity_instruments.equityOption import EquityOption
from instruments.interestRateInstrument.interestRateSwap import InterestRateSwap
from instruments.interestRateInstrument.swaption import Swaption

eq_option = EquityOption(notional=20, S=35, K=35, m=2)
print(str(eq_option))

swap = InterestRateSwap(notional = 2000000, currency = Currency.EUR, timeToSwapStart=0, timeToSwapEnd=4, swapDirection=SwapDirection.RECEIVER)

fw_swap = InterestRateSwap(notional = 1000000,currency = Currency.USD, timeToSwapStart=1, timeToSwapEnd=3, swapDirection = SwapDirection.PAYER)
print(fw_swap)

swaption = Swaption(underlyingSwap=fw_swap,
                    optionMaturity=1,
                    tradeDirection = TradeDirection.SHORT,
                    strikeFixedRate = 0.014,
                    forwardParFixedRate = 0.01)
print(swaption)

{'Instrument': 'EquityOption', 'TradeType': 'Call', 'TradeDirection': 'Long', 'Maturity': 2, 'Startdate': None, 'Notional': 20}
{'Instrument': 'InterestRateSwap', 'TradeType': 'Linear', 'TradeDirection': 'Long', 'Maturity': 3, 'Startdate': 1, 'Notional': 1000000}
{'Instrument': 'Swaption', 'TradeType': 'Call', 'TradeDirection': 'Short', 'Maturity': 3, 'Startdate': 1, 'Notional': 1000000}


## Trade level adjusted notional

Each trade $i$ has a trade level adjusted notional $d_i^a$ assigned to it. This is calculated differently for the different asset classes.

#### Interest rate and credit derivatives

The notional of the trade is usually a well defined value in domestic currency for interest rate and credit derivatives. It is multiplied by a supervisory duration factor. The basic idea is, that the value of the derivative can change more the longer the remaining 

\begin{align*}
d_i &= \text{Notional}_i * SD_i \\
\\
\text{where} \qquad SD_i &=\frac{\exp\left(-0.05 * S_i\right)-\exp\left(-0.05 * E_i\right)}{0.05}
\end{align*}

#### FX derivatives

While the wording in the BCBS paper is a bit more specific we will just assume that every FX traded derivative has a USD leg and set the notional equal the to USD notional.

#### Equity and commodity derivatives

The notional is defined as the price of the underlying. Therefore, it fluctuates over time.

#### Notional of exotic derivatives

For more exotic derivatives which do have adjustable notionals, resetting notionals etc. detailed handling of the notional is defined in paragraph 158.

In [4]:
def trade_level_adjusted_notional(trade: Trade):
    """
    Calculates the trade level adjusted notional as defined in paragraph 157
    
    :param trade: Trade object for which the adjusted notional should be calculated
    :param underlying_price: 
        required for FX trades (exchange rate to domestic currency)
        required for EQ trades (current price of one share)
        required for CO trades (current price of fone unit of commodity)
        not required for IR trades and CR trades
    
    :return: adjusted notional as defined in paragraph 157
    """
    
    if trade.assetClass in (AssetClass.IR, AssetClass.CR):
        sd = (exp(-0.05*trade.s)-exp(-0.05*trade.e))/0.05
        return trade.notional*sd
    
    # This assumes that trade.Notional is defined as quantity of underlying shares for equity and foreign currency notional for FX
    if trade.assetClass == AssetClass.EQ:
        return trade.notional*trade.S
    
    if trade.assetClass == AssetClass.FX:
        return trade.notional

## Supervisory delta adjustments: $\delta_i$

For linear derivatives $\delta$ is 1 for long derivatives and -1 for short derivatives.

For options $\delta$ is defined as under Black-Scholes:

\begin{align*}
\delta_{\text{long Call}} &= +\Phi\left(\frac{\ln\left(P_i / K_i \right) + 0.5 * \sigma_i^2 * T_i}{\sigma_i * \sqrt{T_i}}\right) \\
\\
\text{where} \qquad \Phi &: \text{standard normal cdf} \\
\sigma_i &: \text{supervisory volatility as defined in Table 2 in paragraph 183}
\end{align*}

This delta is multiplied by -1 in case of a long Put option or a short Call option.

No detail is given at this point on the delta calculation of CDO tranches as these are not in the scope of this thesis.

In [5]:
def get_supervisory_volatility(trade: Trade) -> float:
    supervisory_parameter = read_csv('supervisory_parameters.csv')
    sigma = None
    if trade.subClass is None:
        sigma = supervisory_parameter[(supervisory_parameter.AssetClass == trade.assetClass.value)]['SupervisoryOptionVolatility'].iloc[0]
    elif trade.subClass is not None:
        sigma = supervisory_parameter[(supervisory_parameter.AssetClass == trade.assetClass.value) & (supervisory_parameter.SubClass == trade.subClass.value)]['SupervisoryOptionVolatility'].iloc[0]
    return sigma

def calculate_sa_ccr_delta(trade) -> float:
    if (trade.tradeType == TradeType.LINEAR) and (trade.tradeDirection == TradeDirection.LONG):
        return 1
    elif (trade.tradeType == TradeType.LINEAR) and (trade.tradeDirection == TradeDirection.SHORT):
        return -1
    elif trade.tradeType in (TradeType.CALL, TradeType.PUT):
        d1_mult = +1 if trade.tradeType == TradeType.CALL else -1
        
        if trade.tradeType == TradeType.CALL and trade.tradeDirection == TradeDirection.LONG: n_mult = 1
        elif trade.tradeType == TradeType.CALL and trade.tradeDirection == TradeDirection.SHORT: n_mult = -1
        elif trade.tradeType == TradeType.PUT and trade.tradeDirection == TradeDirection.LONG: n_mult = -1
        else: n_mult = +1
        
        sigma = get_supervisory_volatility(trade)
        
        d1 = (log(trade.S/trade.K) + 0.5 * sigma**2 * trade.t)/(sigma*sqrt(trade.t))
        
        delta = n_mult*norm.cdf(d1_mult*d1)
        return delta

Let's try if our delta function works for the different trade types we have set up.

1. An equity option
2. An interest rate swap
3. A forward starting interest rate swap
4. A swaption

In [6]:
print(eq_option)
calculate_sa_ccr_delta(eq_option)

{'Instrument': 'EquityOption', 'TradeType': 'Call', 'TradeDirection': 'Long', 'Maturity': 2, 'Startdate': None, 'Notional': 20}


0.801928045423963

In [7]:
print(swap)
calculate_sa_ccr_delta(swap)

{'Instrument': 'InterestRateSwap', 'TradeType': 'Linear', 'TradeDirection': 'Short', 'Maturity': 4, 'Startdate': 0, 'Notional': 2000000}


-1

In [8]:
print(fw_swap)
calculate_sa_ccr_delta(fw_swap)

{'Instrument': 'InterestRateSwap', 'TradeType': 'Linear', 'TradeDirection': 'Long', 'Maturity': 3, 'Startdate': 1, 'Notional': 1000000}


1

In [9]:
print(swaption)
calculate_sa_ccr_delta(swaption)


{'Instrument': 'Swaption', 'TradeType': 'Call', 'TradeDirection': 'Short', 'Maturity': 3, 'Startdate': 1, 'Notional': 1000000}


-0.33616788698866606

The formula used for the regulatory delta is equivalent to the Black-Scholes formula for delta under the assumption of a risk free rate of 0. As a sanity check we can check if the Delta of a long call is about 0.5 if the call is just about to mature.

In [10]:
eq_option2 = EquityOption(notional = 20, S = 35, K = 35, m = 0.00000001, tradeType=TradeType.CALL, tradeDirection=TradeDirection.LONG)
print(eq_option2)
calculate_sa_ccr_delta(eq_option2)

{'Instrument': 'EquityOption', 'TradeType': 'Call', 'TradeDirection': 'Long', 'Maturity': 1e-08, 'Startdate': None, 'Notional': 20}


0.5000239365368098

Trying to validate methodology by replicating Example 1 of Annex 4a. Setting up trades:

In [11]:
trade1 = InterestRateSwap(notional = 10000000, currency=Currency.USD, timeToSwapStart=0, timeToSwapEnd=10, swapDirection=SwapDirection.PAYER)
trade2 = InterestRateSwap(notional = 10000000, currency=Currency.USD, timeToSwapStart=0, timeToSwapEnd=4, swapDirection=SwapDirection.RECEIVER)
trade3_ul = InterestRateSwap(notional = 5000000, currency=Currency.EUR, timeToSwapStart=1, timeToSwapEnd=11, swapDirection=SwapDirection.RECEIVER)
trade3 = Swaption(underlyingSwap=trade3_ul, optionMaturity=1, tradeDirection=TradeDirection.LONG, strikeFixedRate=0.05, forwardParFixedRate=0.06)

Calculating trade level adjusted notional and supervisory delta. Should be the following:

| Trade   | Adjusted notional (thousands) | Supervisory delta |
|---------|-------------------------------|-------------------|
| Trade 1 | 78694                         | 1                 |
| Trade 2 | 36254                         | -1                |
| Trade 3 | 37428                         | -0.27             |

In [12]:
ind = ['Trade 1', 'Trade 2', 'Trade 3']
data = {'Adjusted notional (thousands)': [round(trade_level_adjusted_notional(trade1)/1000),
                                          round(trade_level_adjusted_notional(trade2)/1000),
                                          round(trade_level_adjusted_notional(trade3)/1000)],
        'Supervisory delta': [round(calculate_sa_ccr_delta(trade1)),
                              round(calculate_sa_ccr_delta(trade2)),
                              round(calculate_sa_ccr_delta(trade3),2)]}
df = DataFrame(data = data, index = ind)
df

,Adjusted notional (thousands),Supervisory delta
Trade 1,78694,1.00
Trade 2,36254,-1.00
Trade 3,37428,-0.27


The implemented formulas correctly replicate the results given in the first Example of Annex 4a.

SA-CCR uses the same Black-Scholes based formula for Swaps as it uses for Equities. It differentiates options in two dimensions. Whether they are *bought* or *sold* and whether they are *Call* or *Put* options (Compare paragraph 159).

SA-CCR defines an option as a call option if it rises in value as the underlying rises in value. A fixed payer swap rises in value as the underlying interest rate rises in value. Therefore, an option to buy a fixed payer swap at a predetermined strike also rises in value as the underlying interest rate rises in value. Therefore, a swaption on a payer swap is considered a *Call* under SA-CCR, while a swaption on a receiver swap is considered a *Put*. 

## Risk horizon

For unmargined transaction the margining factor is


$$MF^{\text{unmargined}}_i = \sqrt{\frac{\min\left(M_i;1\text{ year}\right)}{1\text{ year}}}$$

This factor can be used to scale down a risk weight calibrated for a 1 year horizon to a shorter period.

With margining the margin period of risk (MPOR) is:

* 10 business days for small, uncleared OTC portfolios
* 5 business days for cleared derivatives
* 20 business days for netting sets with more than 5000 transactions that are not with a central counterparty
* and doubling this period for portfolios with outstanding disputes

The margining factor is then

$$ MF^{\text{margined}}_i = \frac{3}{2}\sqrt{\frac{MPOR_i}{1\text{ year}}} $$

At this point we need to introduce a collateral agreement object. For simplicities sake we will not differentiate between collateral and netting sets in this thesis. All trades that are covered by the same collateral agreement are also admissible for netting with each other. (Also refer to the introduction of close out netting above)

The default properties of a collateral agreement are displayed below:

In [13]:
ca = CollateralAgreement()
print(ca)

{
    "margining": "Unmargined",
    "clearing": "Non-centrally cleared derivatives",
    "tradecount": "Under five thousand transactions between the counterparties",
    "dispute": "No margining disputes are outstanding between the counterparties",
    "threshold": 0.0,
    "mta": 0.0,
    "vm": 0.0,
    "posted_im": 0.0,
    "received_im": 0.0,
    "unsegregated_overcollateraliziation_posted": 0.0,
    "unsegregated_overcollateralization_received": 0.0,
    "segregated_overcollateralization_posted": 0.0,
    "segregated_overcollateralization_received": 0.0
}


And they may be changed at the initialization of the collateral agreement or afterwards. The latter case is displayed in the next cell:

In [14]:
ca.margining= Margining.MARGINED
ca.clearing= Clearing.CLEARED
ca.vm = -3000
ca.posted_im = 1000
ca.received_im = 1000

print(ca)

{
    "margining": "Margined",
    "clearing": "Centrally cleared derivatives",
    "tradecount": "Under five thousand transactions between the counterparties",
    "dispute": "No margining disputes are outstanding between the counterparties",
    "threshold": 0.0,
    "mta": 0.0,
    "vm": -3000,
    "posted_im": 1000,
    "received_im": 1000,
    "unsegregated_overcollateraliziation_posted": 0.0,
    "unsegregated_overcollateralization_received": 0.0,
    "segregated_overcollateralization_posted": 0.0,
    "segregated_overcollateralization_received": 0.0
}


With this collateral set object we can define a function for calculation the margining factor:

In [15]:
def margining_factor(trade : Trade, ca : CollateralAgreement) -> float:
    if ca.margining == Margining.UNMARGINED:
        floored_maturity = max(10/250, trade.m) #as Everything is measured in years 10/250 is equal to 10 business days.
        mf_unmargined = sqrt(min(floored_maturity, 1)/1)
        return mf_unmargined
    if ca.margining == Margining.MARGINED:
        # Compare paragraph 164 to see how MPOR needs to be set for margined trades.
        if ca.clearing==Clearing.CLEARED:
            MPOR = 5
        elif ca.tradecount==Tradecount.OVER_FIVE_THOUSAND:
            MPOR = 20
        else: MPOR = 10
        if ca.dispute == Dispute.OUTSTANDING_DISPUTES:
            MPOR = MPOR*2
        
        mf_margined = 3/2 * sqrt(MPOR/250) #Since MPOR above is defined in Business days and not year fractions division by 250 is necessary.
        return mf_margined

For trades of differing maturity let's compare the margining factor for the three most common scenarios:

1. No margining
2. Bilateral uncleared
3. Centrally cleared

In [16]:
one_day = InterestRateSwap(notional = 1000000, currency = Currency.USD, timeToSwapStart=0, timeToSwapEnd=1/250, swapDirection=SwapDirection.PAYER)
two_weeks = InterestRateSwap(notional = 1000000, currency = Currency.USD, timeToSwapStart=0, timeToSwapEnd=10/250, swapDirection=SwapDirection.PAYER)
six_months = InterestRateSwap(notional = 1000000, currency = Currency.USD, timeToSwapStart=0, timeToSwapEnd=125/250, swapDirection=SwapDirection.PAYER)
one_year = InterestRateSwap(notional = 1000000, currency = Currency.USD, timeToSwapStart=0, timeToSwapEnd=250/250, swapDirection=SwapDirection.PAYER)
ten_years = InterestRateSwap(notional = 1000000, currency = Currency.USD, timeToSwapStart=0, timeToSwapEnd=2500/250, swapDirection=SwapDirection.PAYER)

no_margining = CollateralAgreement()
bilateral_margining = CollateralAgreement(margining = Margining.MARGINED)
central_clearing = CollateralAgreement(margining = Margining.MARGINED, clearing = Clearing.CLEARED)

trades = [one_day, two_weeks, six_months, one_year, ten_years]
cas = [no_margining, bilateral_margining, central_clearing]
ar = numpy.empty([3,5])
i = 0
for t in trades:
    j = 0
    for ca in cas:
        ar[j,i] = margining_factor(t,ca)
        j += 1
    i += 1

DataFrame(index=['No margining', 'Bilateral uncleared', 'Centrally cleared'],
          columns=['One day', 'Two weeks', 'Six months', 'One year', 'Ten years'],
          data = ar)

,One day,Two weeks,Six months,One year,Ten years
No margining,0.200000,0.200000,0.707107,1.000000,1.000000
Bilateral uncleared,0.300000,0.300000,0.300000,0.300000,0.300000
Centrally cleared,0.212132,0.212132,0.212132,0.212132,0.212132


## AddOn for interest rate derivatives

#### Step 1 - calculation of effective notional $D_{jk}^{IR}$

\begin{align*}
D_{jk}^{IR} &= \sum_{i\in\left\{Ccy_j, MB_k\right\}}{\delta_i*d_i^{IR}*MF_i}
\end{align*}

Here, the notation $i\in\left\{Ccy_j, MB_k\right\}$ refers to trades whose underlying is the interest rate of a common currency $j$ and which mature in a common maturity bucket $k$

In [17]:
def get_supervisory_factor(assetClass: AssetClass, subClass: SubClass = None) -> float:
    supervisory_parameter = read_csv('supervisory_parameters.csv')
    sf = None
    if subClass is None:
        sf = supervisory_parameter[(supervisory_parameter.AssetClass == assetClass.value)]['Supervisory factor'].iloc[0]
    elif subClass is not None:
        sf = supervisory_parameter[(supervisory_parameter.AssetClass == assetClass.value) & (supervisory_parameter.SubClass == subClass.value)]['Supervisory factor'].iloc[0]
    return sf

def interest_rate_addOn(trades: List[Trade], ca: CollateralAgreement) -> float:
    bucketed_trades = {}
    en_cur_mat = {}
    add_on_cur = Series()
    currencies = set()
    
    for t in trades:
        key = (t.currency, t.get_maturity_bucket())
        currencies.add(t.currency)
        if key in bucketed_trades:
            bucketed_trades[key].append(t)
        else:
            bucketed_trades[key] = [t]
    
    for key, trades in bucketed_trades.items():
        effective_notional = 0
        for t in trades:
            effective_notional += calculate_sa_ccr_delta(t) * trade_level_adjusted_notional(t) * margining_factor(t, ca)
        en_cur_mat[key] = effective_notional
        
    for cur in currencies:
        d_1 = en_cur_mat.get((cur, MaturityBucket.ONE),0)
        d_2 = en_cur_mat.get((cur, MaturityBucket.TWO),0)
        d_3 = en_cur_mat.get((cur, MaturityBucket.THREE),0)
        
        en_cur = sqrt(d_1**2 + d_2**2 + d_3**2 + 1.4*d_1 *d_2 + 1.4*d_2*d_3 + 0.6*d_1*d_3)
        
        add_on_cur[cur] = get_supervisory_factor(assetClass = AssetClass.IR) * en_cur

    return add_on_cur.sum()

For a quick sanity check we can again check against the example 1 in Appendix 4a. The $AddOn_{IR}$ should be 347 thousand USD.

In [18]:
round(interest_rate_addOn([trade1, trade2, trade3], CollateralAgreement())/1000)

347.0

Next, we are setting up a function to calculate the $AddOn$ for derivatives from the FX asset class. The approach is a little simpler than that for the IR asset class as no differentiaten between time buckets is necessary. All derivatives on a common currency pair can be set off against each other.

\begin{align*}
AddOn^{FX} &= \sum_j{AddOn_{HS_j}^{FX}} \\
AddOn^{FX}_{HS_j} &= SF_j^{FX} * | \text{EffectiveNotional}_j^{FX} | \\
\text{EffectiveNotional}_j^{FX} &= \sum_{i \in HS_j}{\delta_i * d_i^{FX} * MF_i^{type}}
\end{align*}

With the hedging sets $i \in HS$ referencing all derivatives on a common currency pair.

In [19]:
def fx_addOn(trades: List[Trade], ca: CollateralAgreement) -> float:
    bucketed_trades = {}
    en_cur = {}
    add_on_cur = Series()
    currencyPairs = set()
    
    for t in trades:
        key = (t.currencyPair)
        currencyPairs.add(t.currencyPair)
        if key in bucketed_trades:
            bucketed_trades[key].append(t)
        else:
            bucketed_trades[key] = [t]
            
    for key, trades in bucketed_trades.items():
        effective_notional = 0
        for t in trades:
            effective_notional += calculate_sa_ccr_delta(t) * trade_level_adjusted_notional(t) * margining_factor(t, ca)
        en_cur[key] = effective_notional
        
    for cur in currencyPairs:
        
        add_on_cur[cur] = get_supervisory_factor(assetClass = AssetClass.FX) * abs(en_cur[cur])
        
    return add_on_cur.sum()

fx_trade1 = FxForward(notional = 1000000, currencyPair = CurrencyPair.GBPUSD, tradeDirection = TradeDirection.LONG, m=3)
fx_trade2 = FxOption(notional= 2000000, currencyPair = CurrencyPair.GBPUSD, tradeDirection = TradeDirection.LONG, tradeType = TradeType.PUT, m = 2, strike = 1.1, currentForwardFxRate=1.07)

fx_addOn([fx_trade1, fx_trade2], CollateralAgreement())

774.974578003427

Since credit and commodity derivatives will not be regarded in this thesis the final $AddOn$ component that we will set up is for the equity asset class. The formula is somwhat simplified since we're only considering single stock options and no index options. With this assumption we can always use the same correlation factor.

In [20]:
def equity_addOn(trades: List[Trade], ca: CollateralAgreement):
    bucketed_trades = {}
    en_eq = {}
    add_on_eq = Series()
    equities = set()
    
    for t in trades:
        key = (t.underlying)
        equities.add(t.underlying)
        if key in bucketed_trades:
            bucketed_trades[key].append(t)
        else:
            bucketed_trades[key] = [t]
            
    for key, trades in bucketed_trades.items():
        effective_notional = 0
        for t in trades:
            effective_notional += calculate_sa_ccr_delta(t) * trade_level_adjusted_notional(t) * margining_factor(t, ca)
        en_eq[key] = effective_notional
        
    for eq in equities:
        
        add_on_eq[eq] = get_supervisory_factor(assetClass = AssetClass.EQ, subClass = EquitySubClass.SINGLE_NAME) * en_eq[eq]
    
    add_on_aggr = sqrt( (0.5*add_on_eq.sum())**2 + (0.75*add_on_eq**2).sum())
    return add_on_aggr

# eq_trade_1 = EquityOption(notional = 20, S = 20 , K =18, m=3)
# eq_trade_2 = EquityOption(notional = 30, S = 18, K = 19, m=3, underlying = Stock.DBK, tradeType=TradeType.PUT)

# equity_addOn([eq_trade_1, eq_trade_2], CollateralAgreement())

Equity add on calculation is validation with one example set in ``test/equity_addOn_validation.xlsx``. The result in the excel is 444.13 USD. Which as can be seen below is in line with the output of the function.

In [21]:
eq_trade1 = EquityOption(notional = 20, S=88, K=91, m=2, tradeDirection= TradeDirection.SHORT, underlying= Stock.ADS)
eq_trade2 = EquityOption(notional = 30, S=28, K=35, m=2, tradeType = TradeType.PUT, tradeDirection = TradeDirection.LONG, underlying = Stock.DBK)
eq_trade3 = EquityForward(notional = 30, m=1.5, S=28, tradeDirection = TradeDirection.LONG, underlying = Stock.DBK)

equity_addOn([eq_trade1, eq_trade2, eq_trade3], CollateralAgreement())

444.13789284343505

Finally we are able to define a function for the PFE that consumes trades and a collateral agreement.

In [22]:
def calculate_pfe(trades: List[Trade], ca : CollateralAgreement):
    ac_bucketing = {}
    addOns = Series()
    V = 0
    for t in trades:
        if t.assetClass in ac_bucketing:
            ac_bucketing[t.assetClass].append(t)
        else:
            ac_bucketing[t.assetClass] = [t]
    for ac, ac_trades in ac_bucketing.items():
        if ac == AssetClass.EQ:
            addOns[ac] = equity_addOn(ac_trades, ca)
        elif ac == AssetClass.IR:
            addOns[ac] = interest_rate_addOn(ac_trades, ca)
        elif ac == AssetClass.FX:
            addOns[ac] = fx_addOn(ac_trades, ca)

    for t in trades:
        pricer_class = t.get_pricer()
        V += pricer_class.price(t)
    
    C = ca.get_C()
    aggregate_addOn = addOns.sum()
    multiplier_var = multiplier(V, C, aggregate_addOn)
    
    PFE = multiplier_var*aggregate_addOn
    return PFE

calculate_pfe([eq_trade1], CollateralAgreement())
calculate_pfe([trade1, trade2, trade3], CollateralAgreement())

346764.3863838184

## Definition of Replacement Cost (RC)

The replacement cost for unmargined trades is defined as 

\begin{align*}
RC &= \max\{V-C;0\} \\
\\
\text{where} \qquad V&: \text{Value of derivative transactions} \\
C&: \text{Net collateral held according to NICA methodology}
\end{align*}

### RC under margining
To consider margining RC is adjusted to be

\begin{align*}
RC &= \max\{V-C; TH+MTA-NICA;0\} \\
\\
\text{where} \qquad TH&: \text{Threshold} \\
MTA&: \text{Minimum Transfer Amount} \\
NICA&: \text{Net Independent Collateral Amount}
\end{align*}

Here, the term $TH+MTA-NICA$ represents the maximum exposure that does not trigger a new variation margin call.

While threshold and the minimum transfer amount are well established parameters of a collateral agreement (compare e.g. Gregory - The XVA Challenge) the net independent collateral amount is a term coined for SA-CCR. 

### Net independendent margin amount - NICA

Independent margin is margin that exceeds the current Variation Margin of the portfolio - this could be a voluntary or accidental overcollateralization or a mandated initial margin amount. If margin is posted to segregated accounts NICA can only become positive.

Margin on a segregated account can surely be recovered if the counterparty to which the margin has been pledged defaults. Therefore this margin can not be lost through a default of the counterparty and can only be used by the counterparty to which it has been plegded in case of a default of the pledgor. Generally speaking VM is transferred to non-segregated accounts while IM is transferred to segregated accounts. It is important to note that over-collateralization or initial margin, sometimes also referred to as *independent amount* that are deposited on an unsegregated account of the counterparty have to be deducted from the NICA.

For bilateral initial margin calculation the regulator requires, that initial margin is payed to segregated accounts. In the case of a bilateral initial margin agreement the NICA would therefore be equal the the IM received from the counterparty i.e. $NICA = IM_{received from Counterparty}$.

On the other hand, there can also be voluntary, one-sided initial margin agreements in place between counterparties that differ drastically in size and default probability. Large investment banks can for example require hedge funds to post initial margin to an unsegregated account of the investment bank to reduce the counterparty credit risk from the investement banks point of view. In this case the NICA would be calculated as $NICA_{\text{Investment Bank}} = IM_{\text{posted by Hedge Fund}}$ while the NICA of the hedge fund would be calculated as $NICA_{\text{Hedge Fund}} = - IM_{\text{posted by Hedge Fund}} $.

If the hedge fund would instead post the IM on a segregated account the NICA of the investment bank remains unchanged while the NICA of the hedge fund would increase to 0. 

Finally, let's set RC up in Python:

In [23]:
def calculate_rc(trades: List[Trade], ca: CollateralAgreement) -> float:
    """
    
    :param v: Current value of the derivative transactions in the netting set
    :param c: Haircut value of the net collateral held
    :param th: Threshold set in the collateral agreement
    :param mta: Minimum transfer amount set in the collateral agreement
    :param nica: Current net independent collateral amount (compare paragraph 143)
    :return: Replacement Cost as defined in paragraph 144
    """
    v = 0
    for t in trades:
        v += t.get_pricer().price(t)
    c = ca.get_C()
    th = ca.threshold
    mta = ca.mta
    nica = ca.get_nica()
    result = max(v-c, th+mta-nica, 0)
    return result

calculate_rc([trade1, trade2, trade3], CollateralAgreement())

60000.0

With all pieces in place we are now able to calculate the EAD according to SA-CCR given a list of trades and a collateral agreement that they are netted and margined under.

In [24]:
trades = [trade1, trade2, trade3]
ca = CollateralAgreement()
calculate_sa_ccr_ead(rc = calculate_rc(trades, ca), pfe = calculate_pfe(trades, ca))

569470.1409373457